## Names: 

#### Mohamed Afif Chifaoui

#### Ricardo Vazquez Alvarez

---

# Linear and Discrete Models First Homework


## Problem statement:

A service company operates in three shifts (Morning, Afternoon, Night) and needs to determine the number of employees assigned to each shift. The objective is to minimize labor costs while ensuring sufficient staffing levels for service demand.

Firstly, we can determine that there are two types of sets for the model (the sets of shifts and the employee types). 

1. $S$: Set of Shifts (Morning, Afternoon, Nights)
2. $E$: Set of employee types (Regular, Part-time, Temporary, On-call, Vacation, Training).

The parameters to be used in the objective function and the constraints will be the following:

1. $C_{e}$: Cost per hour for each employee type. ($e \in E$)
2. $D_{s}$: Service demand for each shift. ($s \in S$).
3. $M_{s}$: Maximum overtime hours allowed for each shift. ($s \in S$)
4. $U_{e}$: Maximum allowed number for each employee type. ($e \in E$, Part-time, Temporary, On-call, Vacation, Training).

The variables are those employees with their type ($e$) to be assigned to a shift ($s$).
- $X_{es}$: Number of employees of type $e$ assigned to shift $s$.

Leading to the Objective function:

$$\text{Minimize} \;\; Z = \sum_{e\in E}^{}\sum_{s\in S}^{} C_{e}\cdot X_{es}$$



### Decision Variables:

The following are the individual decision variables that will be used:

- $x_{1}$: Number of employees assigned to the Morning shift.
- $x_{2}$: Number of employees assigned to the Afternoon shift.
- $x_{3}$: Number of employees assigned to the Night shift.
- $x_{4}$: Overtime hours for the Morning shift.
- $x_{5}$: Overtime hours for the Afternoon shift.
- $x_{6}$: Overtime hours for the Night shift.
- $x_{7}$: Number of employees assigned to the Morning shift.
- $x_{8}$: Number of employees assigned to the Afternoon shift.
- $x_{9}$: Number of employees assigned to the Night shift.
- $x_{10}$: Number of temporary employees assigned to the Morning shift.
- $x_{11}$: Number of temporary employees assigned to the Afternoon shift.
- $x_{12}$: Number of temporary employees assigned to the Night shift.
- $x_{13}$: Number of on-call employees for the Morning shift.
- $x_{14}$: Number of on-call employees for the Afternoon shift.
- $x_{15}$: Number of on-call employees for the Night shift.
- $x_{16}$: Number of employees on vacation during the Morning shift.
- $x_{17}$: Number of employees on vacation during the Afternoon shift.
- $x_{18}$: Number of employees on vacation during the Night shift.
- $x_{19}$: Number of employees on training during the Morning shift.
- $x_{20}$: Number of employees on training during the Afternoon shift.
- $x_{21}$: Number of employees on training during the Night shift.


### Constraints:

1. Shift Coverage Constraints:
$$\sum_{e\in E} X_{es} = D_{s} \qquad \text{for all} \;\;s \in S$$

2. Overtime Constraints:

$$X_{4s} \leq M_{s} \qquad \text{for all} \;\;s \in S$$

3. Employee Type Constraints:

$$X_{7s}, X_{8s}, X_{9s} \leq U_{\text{Part-time}}$$

$$X_{10s}, X_{11s}, X_{12s} \leq U_{\text{Temporary}}$$

$$X_{13s}, X_{14s}, X_{15s} \leq U_{\text{On-call}}$$

$$X_{16s}, X_{17s}, X_{18s} \leq U_{\text{Vacation}}$$

$$X_{19s}, X_{20s} \leq U_{\text{Training}}$$

4. Non-negativity constraints:

$$ X_{es} \geq 0 \quad(\text{for all}\; e\in E,\; s\in S)$$ 


---

### Ignore above

---

In [145]:
import pandas as pd
import numpy as np
from pyomo.environ import * 

In [146]:
model = ConcreteModel()

# Insert your code below
#  'Tuesday', 'Wednesday', 'Thursday', 'Friday'
model.i = Set(initialize = ['Morning', 'Afternoon', 'Night'], doc = 'Shift')

# The types of different Employee
model.j = Set(initialize = ['Regular', 'PartTime', 'Temp', 'OnCall', 'Training', 'Managers', 'Security'], doc = 'EmployeeType')




# total amount of workers needed on a shift

model.total_workers_needed_onshift = Param(model.i, initialize= {'Morning':30, 'Afternoon': 30, 'Night': 30}) # we need 30 workers total
# Total amount of different workers we have for each type (each are independent - a worker of one type cannot be of another type).
model.indiv_workers_have = Param(model.j, initialize={'Regular': 50, 'PartTime': 10, 'Temp': 10, 
                                                      'OnCall': 40, 'Training': 50, 'Managers': 10, 'Security': 5})


# cost of each worker for each shift.
costperworkershift = {

    # total 21 decision variables
    ('Morning', 'Regular'): 30,
    ('Morning', 'PartTime'): 20,
    ('Morning', 'Temp'): 15,
    ('Morning', 'OnCall'): 25,
    ('Morning', 'Training'): 10,
    ('Morning', 'Managers'): 55,
    ('Morning', 'Security'): 40,

    ('Afternoon', 'Regular'): 30,
    ('Afternoon', 'PartTime'): 20,
    ('Afternoon', 'Temp'): 15,
    ('Afternoon', 'OnCall'): 25,
    ('Afternoon', 'Training'): 10,
    ('Afternoon', 'Managers'): 55,
    ('Afternoon', 'Security'): 40,

    ('Night', 'Regular'): 50,
    ('Night', 'PartTime'): 20,
    ('Night', 'Temp'): 15,
    ('Night', 'OnCall'): 30,
    ('Night', 'Training'): 10,
    ('Night', 'Managers'): 60,
    ('Night', 'Security'): 60,
 

}

# Cost function using the above costperworkershift
model.Cost = Param(model.i, model.j, initialize = costperworkershift, doc = 'cost of queries')

In [147]:
# Decision variables we want to solve for
# must be NonNegative since we have that condition (trivial constraint) and must be integers
# (cannot have a decimal amount of a person)

model.x = Var(model.i, model.j, doc = 'Planning', within=NonNegativeIntegers)


# we want to minimize the total cost in a day
def objective_rule(model):
    return sum(model.Cost[i,j]*model.x[i,j] for i in model.i for j in model.j)

model.objective = Objective(rule = objective_rule, sense = minimize, doc = 'cost per day')

In [148]:
# Demand on the shift. Each shift has different demands
def demand(model, i):

    # i = shift, j = employee type 
    # we are adding all the number of employees to be the amount needed on that shift
    return sum(model.x[i,j] for j in model.j)>=model.total_workers_needed_onshift[i]

model.demand = Constraint(model.i, rule = demand, doc= 'employee demand during shift')

In [149]:
# make sure that we are within the constraint of the amount of employees we have
def requisites(model, j):

    # i = shift, j = employee type 
    # we are adding all the number of employees to be the amount needed on that shift
    return sum(model.x[i,j] for i in model.i)<=model.indiv_workers_have[j]

model.reqs = Constraint(model.j, rule = requisites, doc= 'employee of shift type we have')

In [150]:
# If we choose a training, we must also have a regular to accompany and train them (1 regular per training)

def training_regular(model, i):
    return model.x[i, 'Training'] <= model.x[i, 'Regular']

model.train_reg = Constraint(model.i, rule = training_regular, doc = 'Each training accompanied by a Regular')

In [151]:
# We need at least 3 managers at all shifts:
def managers_3(model, i):
    return model.x[i, 'Managers'] >= 3

model.managers_3 = Constraint(model.i, rule = managers_3, doc = 'Must have at least 3 managers')

In [152]:
# We need at least 1 Security per shift
def security(model, i):

    # i = shift, j = employee type
    return model.x[i, 'Security'] >= 1

model.security = Constraint(model.i, rule = security, doc = 'Must have at least 1 Security per shift')

In [153]:

# Define a solver 
Solver = SolverFactory('glpk')

# Obtain the solution
Results = Solver.solve(model)

# Display the solution
model.x.display()

x : Planning
    Size=21, Index=x_index
    Key                       : Lower : Value : Upper : Fixed : Stale : Domain
    ('Afternoon', 'Managers') :     0 :   3.0 :  None : False : False : NonNegativeIntegers
      ('Afternoon', 'OnCall') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
    ('Afternoon', 'PartTime') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
     ('Afternoon', 'Regular') :     0 :  13.0 :  None : False : False : NonNegativeIntegers
    ('Afternoon', 'Security') :     0 :   1.0 :  None : False : False : NonNegativeIntegers
        ('Afternoon', 'Temp') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
    ('Afternoon', 'Training') :     0 :  13.0 :  None : False : False : NonNegativeIntegers
      ('Morning', 'Managers') :     0 :   3.0 :  None : False : False : NonNegativeIntegers
        ('Morning', 'OnCall') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
      ('Morning', 'PartTime') :     0 :   0.0 :  None

In [154]:
# total amount of money spent
model.objective()


2220.0

In [155]:
def obtain_df_solution(model):
    
    names = ['Morning', 'Afternoon', 'Night']
    shifts = ['Regular', 'PartTime', 'Temp', 'OnCall', 'Training', 'Managers', 'Security']
    values = []

    values_array = np.empty((0, 7))

    # appending all the values of how many employee types to get for a shift
    for v in model.component_data_objects(Var, active=True):
        values.append(v.value)

    # morning_nparray = values[0:len(shifts)]
    # afternoon_nparray = values[len(shifts):len(shifts)*2]
    # night_nparray = values[len(shifts)*2, len(shifts)*3]

    for i in range(0,len(names)):
        values_array = np.append(values_array, [values[len(shifts)*i:len(shifts)*(i+1)]], axis=0)

    # creating the df
    df = pd.DataFrame(values_array, index=names, columns=shifts)


    Solver = SolverFactory('glpk')

    # Obtain the solution

    return df

In [156]:
df, objective = obtain_df_solution(model)
print(objective)
df

ValueError: too many values to unpack (expected 2)

In [ ]:
# can be removed

demandonday = [30, 30, 30]
names = ['Morning', 'Afternoon', 'Night']

initializeworkersneeded = {str(key):value for key,value in zip(names,demandonday)}
initializeworkersneeded

{'Morning': 30, 'Afternoon': 30, 'Night': 30}

In [ ]:
def ComputeModel(demandonday:list):

    # demandonday represents a list containing the demands on all shifts per day
    # if we are to use the above list then all we need to change is the
    # model.total_workers_needed_onshift and initialize it with the 
    # specific numbers per shift.

    # Therefore, we need to create a dictionary for this


    # example demandonday = [30, 30, 30]


    # Then create a list of names for the shifts
    names = ['Morning', 'Afternoon', 'Night']

    # create the dictionary needed with a comprehension format
    initializeworkersneeded = {str(key):value for key,value in zip(names,demandonday)}


    model = ConcreteModel()

    # Insert your code below
    #  'Tuesday', 'Wednesday', 'Thursday', 'Friday'
    model.i = Set(initialize = names, doc = 'Shift')

    # The types of different Employee
    model.j = Set(initialize = ['Regular', 'PartTime', 'Temp', 'OnCall', 'Training', 'Managers', 'Security'], doc = 'EmployeeType')

    # total amount of workers needed on a shift
    model.total_workers_needed_onshift = Param(model.i, initialize= initializeworkersneeded) # we need 'a' workers total per shift

    # Total amount of different workers we have for each type (each are independent - a worker of one type cannot be of another type).
    model.indiv_workers_have = Param(model.j, initialize={'Regular': 50, 'PartTime': 10, 'Temp': 10, 
                                                        'OnCall': 40, 'Training': 50, 'Managers': 10, 'Security': 5})


    # cost of each worker for each shift.
    costperworkershift = {

        # total 21 decision variables
        ('Morning', 'Regular'): 30,
        ('Morning', 'PartTime'): 20,
        ('Morning', 'Temp'): 15,
        ('Morning', 'OnCall'): 25,
        ('Morning', 'Training'): 10,
        ('Morning', 'Managers'): 55,
        ('Morning', 'Security'): 40,


        ('Afternoon', 'Regular'): 30,
        ('Afternoon', 'PartTime'): 20,
        ('Afternoon', 'Temp'): 15,
        ('Afternoon', 'OnCall'): 25,
        ('Afternoon', 'Training'): 10,
        ('Afternoon', 'Managers'): 55,
        ('Afternoon', 'Security'): 40,


        ('Night', 'Regular'): 50,
        ('Night', 'PartTime'): 20,
        ('Night', 'Temp'): 15,
        ('Night', 'OnCall'): 30,
        ('Night', 'Training'): 10,
        ('Night', 'Managers'): 60,
        ('Night', 'Security'): 60,

    }

    # Cost function using the above costperworkershift
    model.Cost = Param(model.i, model.j, initialize = costperworkershift, doc = 'cost of queries')

    # Decision variables we want to solve for
    # must be NonNegative since we have that condition (trivial constraint) and must be integers
    # (cannot have a decimal amount of a person)

    model.x = Var(model.i, model.j, doc = 'Planning', within=NonNegativeIntegers)


    # we want to minimize the total cost in a day
    def objective_rule(model):
        return sum(model.Cost[i,j]*model.x[i,j] for i in model.i for j in model.j)

    model.objective = Objective(rule = objective_rule, sense = minimize, doc = 'cost per day')

    def demand(model, i):

        # i = shift, j = employee type 
        # we are adding all the number of employees to be the amount needed on that shift
        return sum(model.x[i,j] for j in model.j)>=model.total_workers_needed_onshift[i]

    model.demand = Constraint(model.i, rule = demand, doc= 'employee demand during shift')

    def requisites(model, j):

        # i = shift, j = employee type 
        # we are adding all the number of employees to be the amount needed on that shift
        return sum(model.x[i,j] for i in model.i)<=model.indiv_workers_have[j]

    model.reqs = Constraint(model.j, rule = requisites, doc= 'employee of shift type we have')

    # If we choose a training, we must also have a regular to accompany and train them (1 regular per training)

    def training_regular(model, i):

        return model.x[i, 'Training'] <= model.x[i, 'Regular']

    model.train_reg = Constraint(model.i, rule = training_regular, doc = 'Each training accompanied by a Regular')

    # We need at least 3 managers at all shifts:
    def managers_3(model, i):
        return model.x[i, 'Managers'] >= 3

    model.managers_3 = Constraint(model.i, rule = managers_3, doc = 'Must have at least 3 managers')

    # We need at least 1 Security per shift
    def security(model, i):

        # i = shift, j = employee type
        return model.x[i, 'Security'] >= 1

    model.security = Constraint(model.i, rule = security, doc = 'Must have at least 1 Security per shift')

    Results = Solver.solve(model)
    
    obj = model.objective()

    
    return model,obj

In [ ]:
demandMonday  = [30,30,30]
modelMonday,objectiveMonday = ComputeModel(demandMonday)
dfMonday = obtain_df_solution(modelMonday)


demandTuesday= [60,50,50]
modelTuesday, objectiveTuesday  = ComputeModel(demandTuesday)
dfTuesday= obtain_df_solution(modelTuesday)


# pricing_of_each

In [ ]:
selected_row = dfMonday.iloc[1]
selected_row = [each for each in selected_row]
selected_row

[13.0, 0.0, 0.0, 0.0, 13.0, 3.0, 1.0]

In [ ]:
# note that demand on day must be within the limits of how many employees we actually have

# demandonday = [50,60,50]

# mymodel = ComputeModel(demandonday)


# df, objective = obtain_df_solution(mymodel)
# print(objective)
# df

# demandmonday = [20,20,20]
# demandtuesday = [20,20,20]
# demandwednesday = [20,20,20]
# demandthursday = [20,20,20]
# demandfriday = [20,20,20]
# demandsaturday = [20,20,20]
# demandsunday = [20,20,20]

# demandtotal = [demandmonday, demandtuesday, demandwednesday, demandthursday, demandfriday, demandsaturday, demandsunday]


# alldfs = []


# for i in range(0,len(demandtotal)):

#     mymodel = ComputeModel(demandtotal[i])

#     df, objective = obtain_df_solution(mymodel)
#     alldfs.append(df)






# 




---
---

# Todo


- Even out the cost of each employee per shift


- Plot for each day of the week a calender for the employees


## Part C: Sensitivities associated with each constraint, and interpretion of the obtained values.

In [ ]:
# # Display the optimal solution
# print(f"Optimal Value: {model.objective()}")

# # Sensitivity analysis
# print("\nSensitivity Analysis:")
# # Sensitivity analysis using solver results
# print("\nSensitivity Analysis:")
# dual_values = {}
# for constr in model.component_objects(Constraint, active=True):
#     dual_values[constr] = model.dual[constr]

# print("Dual Values:", dual_values)

In [ ]:
# from pyomo.contrib.sensitivity_toolbox.sens import sensitivity_calculation

In [ ]:
# model.pert_workneeded = Param(model.i, initialize= {'Morning':40, 'Afternoon': 50, 'Night': 60}) # we need 30 workers total
# # model.perturbed_eta1 = Param(initialize = 4.0)

# # model.perturbed_eta2 = Param(initialize = 1.0)
# m_kaug_dsdp = sensitivity_calculation('k_aug', model, [model.total_workers_needed_onshift], [model.pert_workneeded], tee=False)

In [ ]:
# model.pprint()

In [ ]:
# # Create a pandas DataFrame from the sensitivity data
# sensitivity_df = pd.DataFrame.from_dict({
#     'Variable': [var for var in model.component_objects(Var, active=True)],
#     'Reduced Cost': [model.rc[var] for var in model.component_objects(Var, active=True)],
#     'Dual Value': [model.dual[constr] for constr in model.component_objects(Constraint, active=True)]
# })

# # Display the sensitivity DataFrame
# print("\nSensitivity Analysis:")
# print(sensitivity_df)

### Part D: Binary Variables:

Modify the problem in (a) to impose some logical and conditional constraints that require the use of binary or integer variables. The more integer variables or constraints, the better. Implement and solve this new model and interpret the results.

 Employee | Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday | Profit
 -------|--------|--|--|---------
  1 |  9 | 9 | 9  | |6  |7   8|| 
  2 |  4 | 7 | 10 | 40 |
  3 |  2 | 3 | 4  | 5 | 6 | 7 | 8|   4 |  7 | 4 | 1  |  15
  4 |  2 | 3 | 4  | 5 | 6 | 7 | 8| 
  5 |  2 | 3 | 4  | 5 | 6 | 7 | 8| 
  6 |  2 | 3 | 4  | 5 | 6 | 7 | 8| 
1 |  2 | 3 | 4  | 5 | 6 | 7 | 8| 



| $_\text{Employee Type}$  $^{\text{Day of week}}$ | Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday | Profit |
|---------------|---------|-----------|-------|-----|-----|-----|-----|-----|
| 1- Regular       | 15 | 15 | 15 |15 | 20 | 20 | 20 | 200 |  
| 2- Part-time     | 10 | 10 | 10 | 10 | 15 | 15 | 15 | 130|  
| 3- Temporary     | 8 | 8 | 8 | 8 | 12 | 12 | 12 | 115 | 
| 4- On-call       | 9 | 9 | 9 | 9 | 13 | 13 | 13 | 95 |  
| 5- Training      | 4 | 4 | 4 |4 |8 | 8 | 8 | 80 |  
| 6- Manager       | 18 | 18 | 18 | 18 | 18 | 23 | 23 | 225|    


In [160]:
#


from pyomo.environ import *
import gurobipy
model = ConcreteModel()

# variables
model.x = Var([1,2,3,4,5,6], domain=Binary) # NOTE the domain = Binary

# objective function --> DONT FORGET THE sense=maximize
model.OBJ = Objective(expr=200*model.x[1] + 130*model.x[2] + 115*model.x[3] + 95*model.x[4] + 80*model.x[5] + 225*model.x[6], sense=maximize)

# constraints

# constraint for each days of the week 1-7(Mon-Sun)

# Monday
model.cons1 = Constraint(expr=15*model.x[1] + 10*model.x[2] + 8*model.x[3] + 9*model.x[4] + 4*model.x[5] + 20*model.x[6] <=50)
# Tuesday
model.cons2= Constraint(expr=15*model.x[1] + 10*model.x[2] + 8*model.x[3] + 9*model.x[4] + 4*model.x[5]+ 20*model.x[6] <=50)
# Wednesday
model.cons3 = Constraint(expr=15*model.x[1] + 10*model.x[2] + 8*model.x[3] + 9*model.x[4] + 4*model.x[5]+ 20*model.x[6] <=50)
# Thursday
model.cons4 = Constraint(expr=15*model.x[1] + 10*model.x[2] + 8*model.x[3] + 9*model.x[4] + 4*model.x[5]+ 20*model.x[6] <=50)
# Friday
model.cons5 = Constraint(expr=20*model.x[1] + 15*model.x[2] + 12*model.x[3] + 13*model.x[4] + 8*model.x[5]+ 25*model.x[6] <=80)
# Saturday
model.cons6 = Constraint(expr=20*model.x[1] + 15*model.x[2] + 12*model.x[3] + 13*model.x[4] + 8*model.x[5]+ 25*model.x[6] <=80)
# Sunday
model.cons7 = Constraint(expr=20*model.x[1] + 15*model.x[2] + 12*model.x[3] + 13*model.x[4] + 8*model.x[5]+ 25*model.x[6] <=80)




Solver = SolverFactory('glpk')

# Solver = SolverFactory('gurobi')

Results = Solver.solve(model)

# Display solution
display(model.OBJ)


OBJ : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 635.0


In [144]:
display(model)

Model unknown

  Variables:
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   1.0 :     1 : False : False : Binary
          2 :     0 :   1.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   0.0 :     1 : False : False : Binary
          5 :     0 :   1.0 :     1 : False : False : Binary
          6 :     0 :   1.0 :     1 : False : False : Binary

  Objectives:
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 635.0

  Constraints:
    cons1 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 49.0 :  50.0
    cons2 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 49.0 :  50.0
    cons3 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 49.0 :  50.0
    cons4 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 49.0 :  50.0
    cons5 : Size=1

# Bibliography